In [7]:
import os
import sys
import pickle
import numpy as np
import networkx as nx

sys.stdout.flush()

# 将子目录添加到 sys.path
current_dir = os.getcwd()
sys.path.append(os.path.join(current_dir,'EffectiveResistanceSampling'))
from EffectiveResistanceSampling.Network import *

sys.path.append(os.path.join(current_dir,'utilities'))
from utilities.tools import *

In [8]:
import pickle
from clusim.clustering import Clustering
from sklearn.metrics import normalized_mutual_info_score
import clusim.sim as sim

def similarity_score(mu, graph_type, delete_type, embedding_method, percent):
    """Process a specific mixing parameter (mu) to do community detection."""
    mu_str = f"{mu:.2f}"
    detected_dir = f'communitydetection_{delete_type}'
    detected_dir = "leiden_" + detected_dir
    if delete_type=="original":
        raw_euclid_path = f'{detected_dir}/{graph_type}_{delete_type}_{embedding_method}_euclid_mu{mu_str}.pkl'
    else:
        raw_euclid_path = f'{detected_dir}_{percent}/{graph_type}_{delete_type}_{embedding_method}_euclid_mu{mu_str}.pkl'
    with open(raw_euclid_path, 'rb') as file:
        euclid_membership = pickle.load(file)
    _, original_membership = load_graph(mu, graph_type, "original", percent)

    raw_score_mu = np.zeros((len(euclid_membership),2))
    for i in range(len(euclid_membership)):
        raw_score_mu[i,0] = normalized_mutual_info_score(original_membership[i], euclid_membership[i], average_method='arithmetic')

        euclid_membership_clus = Clustering({j: [euclid_membership[i][j]] for j in range(len(euclid_membership[i]))})
        original_membership_clus = Clustering({j: [original_membership[i][j]] for j in range(len(original_membership[i]))})
        
        raw_score_mu[i,1] = sim.element_sim(original_membership_clus, euclid_membership_clus, alpha=0.9)

    #print(raw_score_mu)

    if delete_type=="original":
        output_dir = f'leiden_results_{delete_type}'
    else:
        output_dir = f'leiden_results_{delete_type}_{percent}'
    os.makedirs(output_dir, exist_ok=True)
    raw_score_path = f'{output_dir}/{graph_type}_{delete_type}_{embedding_method}_raw_score_mu{mu_str}.pkl'
    with open(raw_score_path, 'wb') as file:
        pickle.dump(raw_score_mu, file)
    # print(f"RAW_SCORE for mu={mu_str} saved to {raw_score_path}")

In [9]:
'''
mu=0.05
graph_type = 'lfr'
delete_type = 'original'
embedding_method = "lap" # "lap" or "n2v"
percent = 0.6
similarity_score(mu, graph_type, delete_type, embedding_method, percent)
'''

'\nmu=0.05\ngraph_type = \'lfr\'\ndelete_type = \'original\'\nembedding_method = "lap" # "lap" or "n2v"\npercent = 0.6\nsimilarity_score(mu, graph_type, delete_type, embedding_method, percent)\n'

In [10]:
start_step = 0.05
step_size = 0.05
graph_type = "ppm"
end_steps = {"lfr": 0.5, "ppm": 0.9}
end_step = end_steps[graph_type]
MU = np.around(np.arange(start_step, end_step + 0.01, step_size), decimals=2)
percent = 0.6

graph_types = ['lfr', 'ppm']
delete_types = ['original', 'random', 'sparse']
#delete_types = ['random', 'sparse']
#delete_types = ['sparse']
embedding_method = 'lap'


for graph_type in graph_types:
    for delete_type in delete_types:
        end_step = end_steps[graph_type]
        MU = np.around(np.arange(start_step, end_step + 0.01, step_size), decimals=2)
        for mu in MU:
            similarity_score(mu, graph_type, delete_type, embedding_method, percent)
            print(f"Work Done for {graph_type}, {delete_type}, mu={mu}")

Work Done for lfr, original, mu=0.05
Work Done for lfr, original, mu=0.1
Work Done for lfr, original, mu=0.15
Work Done for lfr, original, mu=0.2
Work Done for lfr, original, mu=0.25
Work Done for lfr, original, mu=0.3
Work Done for lfr, original, mu=0.35
Work Done for lfr, original, mu=0.4
Work Done for lfr, original, mu=0.45
Work Done for lfr, original, mu=0.5
Work Done for lfr, random, mu=0.05
Work Done for lfr, random, mu=0.1
Work Done for lfr, random, mu=0.15
Work Done for lfr, random, mu=0.2
Work Done for lfr, random, mu=0.25
Work Done for lfr, random, mu=0.3
Work Done for lfr, random, mu=0.35
Work Done for lfr, random, mu=0.4
Work Done for lfr, random, mu=0.45
Work Done for lfr, random, mu=0.5
Work Done for lfr, sparse, mu=0.05
Work Done for lfr, sparse, mu=0.1
Work Done for lfr, sparse, mu=0.15
Work Done for lfr, sparse, mu=0.2
Work Done for lfr, sparse, mu=0.25
Work Done for lfr, sparse, mu=0.3
Work Done for lfr, sparse, mu=0.35
Work Done for lfr, sparse, mu=0.4
Work Done for 